In [1]:
import re # La utilizo para realizar búsquedas y manipulaciones avanzadas de patrones de texto
import chardet # La utilizamos para detectar la codificación de caracteres en texto
import ast # La utilizo para analizar y manipular expresiones y declaraciones
import json #  La utilizo para trabajar con datos en formato JSON 
import pandas as pd

Dateccion de codificacion del archivo


In [3]:
def detectar_encoding(archivo):
    with open(archivo, 'rb') as f:
        resultado=chardet.detect(f.read())
        encoding= resultado['encoding']
        confianza=resultado['confidence']
        return (f'El encoding detectado es {encoding} con una confianza de {confianza*100:.2f}%')


AUSTRALIAN_USER_REVIEWS

In [4]:
archivo='australian_user_reviews.json'
detectar_encoding(archivo)

'El encoding detectado es MacRoman con una confianza de 69.75%'

Ahora se itera el archivo para construir un dataframe.

In [5]:
rows = []
with open ('australian_user_reviews.json', encoding='MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

df_reviews = pd.DataFrame(rows)
df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [6]:
df_reviews.shape

(25799, 3)

Se define funcion para detectar columnas anidadas

In [3]:
def anidada_True(df):
    anidadas=[]
    for columna in df.columns:
        if df[columna].apply(lambda x: isinstance(x,(list,dict))).any():
            anidadas.append(columna)
    return (f'La/s columna/s con datos anidados es/son: {anidadas}')

In [8]:
anidada_True(df_reviews)

"La/s columna/s con datos anidados es/son: ['reviews']"

Como la columna anidada contiene diccionarios:

In [9]:
df_reviews['reviews']=df_reviews['reviews'].apply(lambda x: x[0] if isinstance(x, list) and len(x)>0 else {})

In [10]:
reviews_desanidado=df_reviews.join(pd.json_normalize(df_reviews['reviews']))

In [11]:
reviews_desanidado.head(2)

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...


Se elimina la columna reviews

In [16]:
reviews_desanidado=reviews_desanidado.drop(columns=['reviews'], axis=1)

In [17]:
reviews_desanidado.head(3)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...


In [18]:
reviews_desanidado.tail(3)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,1 person found this review funny,Posted June 23.,,570,1 of 1 people (100%) found this review helpful,True,Well Done
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 21.,,233270,No ratings yet,True,this is a very fun and nice 80s themed shooter...
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...


In [ ]:
reviews_desanidado['funny']

In [19]:
reviews_desanidado.shape

(25799, 9)

In [91]:
item=reviews_desanidado['item_id'].dropna()

In [20]:
reviews_desanidado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      25799 non-null  object
 1   user_url     25799 non-null  object
 2   funny        25771 non-null  object
 3   posted       25771 non-null  object
 4   last_edited  25771 non-null  object
 5   item_id      25771 non-null  object
 6   helpful      25771 non-null  object
 7   recommend    25771 non-null  object
 8   review       25771 non-null  object
dtypes: object(9)
memory usage: 1.8+ MB


Se guarda la primera base de datos desanidad en un csv

In [25]:
reviews_desanidado.to_csv('aus_reviews.csv', index=False)

In [21]:
print(reviews_desanidado.isna().sum())

user_id         0
user_url        0
funny          28
posted         28
last_edited    28
item_id        28
helpful        28
recommend      28
review         28
dtype: int64


In [22]:
print(reviews_desanidado.isnull().sum())

user_id         0
user_url        0
funny          28
posted         28
last_edited    28
item_id        28
helpful        28
recommend      28
review         28
dtype: int64


Se re-confirma que los 28 datos faltantes sean correspondientes en termonios de las observaciones (filas)

In [23]:
reviews_filtrado=reviews_desanidado[reviews_desanidado['funny'].isna()]

In [24]:
reviews_filtrado

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
62,gdxsd,http://steamcommunity.com/id/gdxsd,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,76561198094224872,http://steamcommunity.com/profiles/76561198094...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1047,76561198021575394,http://steamcommunity.com/profiles/76561198021...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3954,cmuir37,http://steamcommunity.com/id/cmuir37,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5394,Jaysteeny,http://steamcommunity.com/id/Jaysteeny,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6135,ML8989,http://steamcommunity.com/id/ML8989,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7583,76561198079215291,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7952,76561198079342142,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9894,76561198061996985,http://steamcommunity.com/profiles/76561198061...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10381,76561198108286351,http://steamcommunity.com/profiles/76561198108...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


las filas con valores nuleos son 28 lo que representa el 0.1%, por ende se descartan estas 28 observaciones

In [44]:
reviews_desanidado_nona=reviews_desanidado.dropna()

In [45]:
reviews_desanidado_nona.shape

(25771, 9)

In [ ]:
reviews_desanidado_nona['funny']=reviews_desanidado_nona['funny'].replace('','0')

In [66]:
reviews_desanidado_nona.head()

,user_id,user_url,funny,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,0,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,0,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,0,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3,35 of 43 people (81%) found this review helpful,True,Git gud


In [61]:
reviews_desanidado_nona['funny'].unique()

array(['0', '3', '1', '5', '17', '2', '524', '4', '12', '6', '103', '16',
       '34', '7', '9', '45', '207', '718', '23', '100', '11', '19', '32',
       '44', '10', '20', '36', '15', '131', '279', '122', '269', '296',
       '46', '8', '42', '80', '28', '417', '13', '14', '133', '115', '72',
       '260', '405', '47', '1130', '856', '71', '30', '89', '75', '518',
       '719', '82', '49', '167', '77', '60', '26', '675', '22', '99',
       '120', '211', '27', '21', '228', '25', '76', '56', '85', '40',
       '203', '480', '39', '759', '53', '38', '52', '70', '429', '18',
       '69', '24', '41', '194', '124', '450', '607', '946', '50', '142',
       '181', '171', '267', '139', '161', '1284', '127', '155', '191',
       '51', '461', '94', '841', '2598', '57', '180', '502', '215', '830',
       '327', '185', '255', '469', '1002', '198', '31', '214', '116',
       '534', '33', '354', '128', '396', '59', '67', '162', '37', '284',
       '1954', '276', '35', '29', '651', '83', '104', '1100

Se retira, el texto, las celdas sin registro se cambian a cero, se retiran las comas y se convierte a tipo entero como la cantidad que encontraron divertido el comentario-

In [48]:
reviews_desanidado_nona.loc[reviews_desanidado_nona['funny'].str.contains(' people found this review funny', case=False), 'funny']=reviews_desanidado_nona['funny'].str.replace(' people found this review funny','', case=False)

In [50]:
reviews_desanidado_nona.loc[reviews_desanidado_nona['funny'].str.contains(' person found this review funny', case=False), 'funny']=reviews_desanidado_nona['funny'].str.replace(' person found this review funny','', case=False)

In [53]:
reviews_desanidado_nona.loc[reviews_desanidado_nona['funny'].str.contains(',', case=False), 'funny']=reviews_desanidado_nona['funny'].str.replace(',','', case=False)

In [56]:
funny=reviews_desanidado_nona['funny'].astype(int)

In [62]:
reviews_desanidado_nona['funny']=funny

C:\Users\57319\AppData\Local\Temp\ipykernel_13852\712542829.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_desanidado_nona['funny']=funny


In [60]:
funny.sum()

60061

se retiran las colunas que no se utilizaran para las consultas

In [65]:
reviews_desanidado_nona=reviews_desanidado_nona.drop(columns=['posted','last_edited','user_url'])

Ahora se analiza 'helpful' referente a que tan util fue la opinion

In [68]:
reviews_desanidado_nona['helpful'].unique()

array(['No ratings yet',
       '15 of 20 people (75%) found this review helpful',
       '2 of 2 people (100%) found this review helpful',
       '35 of 43 people (81%) found this review helpful',
       '7 of 8 people (88%) found this review helpful',
       '0 of 1 people (0%) found this review helpful',
       '3 of 3 people (100%) found this review helpful',
       '0 of 2 people (0%) found this review helpful',
       '2 of 4 people (50%) found this review helpful',
       '1 of 3 people (33%) found this review helpful',
       '28 of 31 people (90%) found this review helpful',
       '6 of 9 people (67%) found this review helpful',
       '1 of 2 people (50%) found this review helpful',
       '3 of 5 people (60%) found this review helpful',
       '4 of 5 people (80%) found this review helpful',
       '0 of 8 people (0%) found this review helpful',
       '27 of 41 people (66%) found this review helpful',
       '1 of 1 people (100%) found this review helpful',
       '4 of 4 

La proporción de personas que encontraron útil la opinión esta dada sobre cada item, mas no sobre el total de personas que la leyeron, entonces podría resultar útil tomar términos absolutos (quienes encontraron útil la opinión) y descartar términos relativos.

In [71]:
reviews_desanidado_nona.loc[reviews_desanidado_nona['helpful'].str.contains('No ratings yet', case=False), 'helpful']=reviews_desanidado_nona['helpful'].str.replace('No ratings yet','0', case=False)

In [72]:
reviews_desanidado_nona.head()

,user_id,user_url,funny,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,0,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,0,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,0,0,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,0,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3,35 of 43 people (81%) found this review helpful,True,Git gud


In [73]:
reviews_desanidado_nona['helpful']=reviews_desanidado_nona['helpful'].apply(lambda x: x.split(' ')[0])

Ahora se retiran las comas

In [77]:
reviews_desanidado_nona['helpful']=reviews_desanidado_nona['helpful'].str.replace(',','')

Ahora se pasa a tipo entero

In [78]:
helpful=reviews_desanidado_nona['helpful'].astype(int)

In [79]:
reviews_desanidado_nona['helpful']=helpful

In [80]:
reviews_desanidado_nona.head()

,user_id,user_url,funny,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,0,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,0,15,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,0,0,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,0,2,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3,35,True,Git gud


La variable recommend se convierte a una variable binaria.

In [87]:
reviews_desanidado_nona['recommend']=reviews_desanidado_nona['recommend'].astype(int)

In [88]:
reviews_desanidado_nona.head()

,user_id,user_url,funny,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,0,1,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,0,15,1,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,0,0,1,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,0,2,1,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3,35,1,Git gud


In [89]:
reviews_desanidado_nona=reviews_desanidado_nona.drop(columns=['user_url'])

In [96]:
reviews_desanidado_nona.head()

,user_id,funny,item_id,helpful,recommend,review
0,76561197970982479,0,1250,0,1,Simple yet with great replayability. In my opi...
1,js41637,0,251610,15,1,I know what you think when you see this title ...
2,evcentric,0,248820,0,1,A suitably punishing roguelike platformer. Wi...
3,doctr,0,250320,2,1,This game... is so fun. The fight sequences ha...
4,maplemage,3,211420,35,1,Git gud


Se importa con csv el dataframe

In [97]:
reviews_desanidado_nona.to_csv('readyfor_sentiment_analysis.csv', index=False)

In [98]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

Se descargan los datos necesarios para VADER

In [99]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\57319\AppData\Roaming\nltk_data...


True

Inicia el analizador de sentimientos

In [100]:
sia = SentimentIntensityAnalyzer()

In [101]:
def sentiment_analysis(text):
    score = sia.polarity_scores(text)['compound']
    if score >= 0.05:
        return 2  
    elif score <= -0.05:
        return 0  
    else:
        return 1  

In [102]:
reviews_desanidado_nona['sentiment_analysis']=reviews_desanidado_nona['review'].apply(sentiment_analysis)

In [104]:
reviews_desanidado_nona.head()

,user_id,funny,item_id,helpful,recommend,review,sentiment_analysis
0,76561197970982479,0,1250,0,1,Simple yet with great replayability. In my opi...,2
1,js41637,0,251610,15,1,I know what you think when you see this title ...,2
2,evcentric,0,248820,0,1,A suitably punishing roguelike platformer. Wi...,2
3,doctr,0,250320,2,1,This game... is so fun. The fight sequences ha...,2
4,maplemage,3,211420,35,1,Git gud,1


In [105]:
reviews_desanidado_nona=reviews_desanidado_nona.drop(columns=['review'])

In [106]:
reviews_desanidado_nona.head()

,user_id,funny,item_id,helpful,recommend,sentiment_analysis
0,76561197970982479,0,1250,0,1,2
1,js41637,0,251610,15,1,2
2,evcentric,0,248820,0,1,2
3,doctr,0,250320,2,1,2
4,maplemage,3,211420,35,1,1


In [108]:
reviews_desanidado_nona.to_csv('readywith_sentiment_analysis.csv', index=False)

In [107]:
reviews_desanidado_nona.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25771 entries, 0 to 25798
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             25771 non-null  object
 1   funny               25771 non-null  int32 
 2   item_id             25771 non-null  object
 3   helpful             25771 non-null  int32 
 4   recommend           25771 non-null  int32 
 5   sentiment_analysis  25771 non-null  int64 
dtypes: int32(3), int64(1), object(2)
memory usage: 2.1+ MB


Aplicando a la columna objetivo

OUTPUT_STEAM_GAMES

Deteccion de codificacion

In [7]:
archivo1='output_steam_games.json'
detectar_encoding(archivo1)

'El encoding detectado es ascii con una confianza de 100.000000%'

Con pandas se puede convertir a dataframe de manera directa

In [6]:
df_games = pd.read_json('output_steam_games.json', lines=True)
df_games.head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [7]:
df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


ahora, facilita el manejo del archivo pasarlo a csv

In [9]:
df_games.to_csv('output_games.csv', index=False)

varificacion de clumnas anidadas:

In [8]:
anidada_True(df_games)

"La/s columna/s con datos anidados es/son: ['genres', 'tags', 'specs']"

In [9]:
import numpy as np
import gzip
import re
import ast


In [11]:
df_games.shape

(120445, 13)

In [10]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [12]:
print(df_games.isna().sum())

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64


Hay una gran cantidad de filas vacias, se procede a eliminar las que esten completamente vacias


In [20]:
filas_vacias = df_games.isnull().all(axis=1).sum()
print(f"Filas completamente vacías: {filas_vacias} es decir un 73%" )

Filas completamente vacías: 88310 es decir un 73%


In [17]:
output_games=df_games.dropna(how='all')

In [18]:
output_games.shape

(32135, 13)

In [22]:
output_games.isnull().sum()

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

Se eliminan las columna inecesarias

In [23]:
output_games.drop(['publisher','url','specs','early_access','reviews_url'],axis=1,inplace=True)

C:\Users\57319\AppData\Local\Temp\ipykernel_6952\529313262.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_games.drop(['publisher','url','specs','early_access','reviews_url'],axis=1,inplace=True)


In [28]:
output_games.isnull().sum()

genres          3144
app_name           0
title           2047
release_date    2065
tags               0
price           1344
id                 0
developer       3270
dtype: int64

Dado que el juego que no registra nombre, id o tag, no es utili para un sistema de recomendacion, se descatan las filas donde estos datos faltan.

In [26]:
output_games.dropna(subset=['app_name','tags','id'],inplace=True)

C:\Users\57319\AppData\Local\Temp\ipykernel_6952\784636219.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_games.dropna(subset=['app_name','tags','id'],inplace=True)


In [27]:
output_games.head()

,genres,app_name,title,release_date,tags,price,id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140.0,Kotoshiro
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",Free To Play,643980.0,Secret Level SRL
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",Free to Play,670290.0,Poolians.com
88313,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.99,767400.0,彼岸领域
88314,None,Log Challenge,None,None,"[Action, Indie, Casual, Sports]",2.99,773570.0,None


In [30]:
output_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31970 entries, 88310 to 120444
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28826 non-null  object 
 1   app_name      31970 non-null  object 
 2   title         29923 non-null  object 
 3   release_date  29905 non-null  object 
 4   tags          31970 non-null  object 
 5   price         30626 non-null  object 
 6   id            31970 non-null  float64
 7   developer     28700 non-null  object 
dtypes: float64(1), object(7)
memory usage: 2.2+ MB


Para el caso especifico del precio, se llenaran los faltantes con el precio promedio

In [31]:
output_games['price'].unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 2.99, 3.99, 9.99,
       18.99, 29.99, None, 'Free', 10.99, 1.5899999999999999, 14.99, 1.99,
       59.99, 8.99, 6.99, 7.99, 39.99, 19.99, 7.49, 12.99, 5.99, 2.49,
       15.99, 1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99,
       16.99, 79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 109.99, 149.99,
       771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 64.99,
       'Free Mod', 54.99, 74.99, 'Install Theme', 0.89, 'Third-party',
       0.5, 'Play Now', 299.99, 1.29, 119.99, 3.0, 15.0, 5.49, 23.99,
       49.0, 20.99, 10.93, 1.3900000000000001,
       'Free HITMAN™ Holiday Pack', 36.99, 4.49, 2.0, 4.0, 9.0, 234.99,
       1.9500000000000002, 1.5, 199.0, 189.0, 6.66, 27.99, 10.49, 129.99,
       179.0, 26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33, 199.99,
       22.99, 320.0, 38.85, 71.7, 59.95, 995.0, 27.49,

Se descartan las demas filas vacias

In [41]:
output_games.dropna(subset=['genres','release_date','price','developer'],inplace=True)

C:\Users\57319\AppData\Local\Temp\ipykernel_6952\478003002.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_games.dropna(subset=['genres','release_date','price','developer'],inplace=True)


In [42]:
output_games.isnull().sum()

genres          0
app_name        0
title           0
release_date    0
tags            0
price           0
id              0
developer       0
dtype: int64

In [43]:
output_games.head()

,genres,app_name,title,release_date,tags,price,id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140.0,Kotoshiro
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",0,643980.0,Secret Level SRL
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",Free to Play,670290.0,Poolians.com
88313,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.99,767400.0,彼岸领域
88315,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",3.99,772540.0,Trickjump Games Ltd


Se descarta 'title' puesto que es igual a 'app name'

In [44]:
output_games.drop('title',axis=1,inplace=True)

C:\Users\57319\AppData\Local\Temp\ipykernel_6952\2707288407.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_games.drop('title',axis=1,inplace=True)


In [45]:
output_games['id'].duplicated().sum()

1

Ids duplicados se descartan

In [46]:
output_games.drop_duplicates(subset='id',inplace=True)

C:\Users\57319\AppData\Local\Temp\ipykernel_6952\646411544.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_games.drop_duplicates(subset='id',inplace=True)


In [47]:
output_games['id'].duplicated().sum()

0

In [50]:
output_games.head(50)

,genres,app_name,release_date,tags,price,id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140.0,Kotoshiro
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",0,643980.0,Secret Level SRL
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",Free to Play,670290.0,Poolians.com
88313,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.99,767400.0,彼岸领域
88315,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",3.99,772540.0,Trickjump Games Ltd
88316,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Basic Pass,2018-01-04,"[Free to Play, Indie, Simulation, Sports]",9.99,774276.0,Poppermost Productions
88317,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Pro Pass,2018-01-04,"[Free to Play, Indie, Simulation, Sports]",18.99,774277.0,Poppermost Productions
88318,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Legend Pass,2018-01-04,"[Free to Play, Indie, Simulation, Sports]",29.99,774278.0,Poppermost Productions
88322,"[Action, Adventure, Casual, Indie, RPG]",Army of Tentacles: (Not) A Cthulhu Dating Sim:...,2018-01-04,"[Action, Adventure, RPG, Indie, Casual]",10.99,770380.0,Stegalosaurus Game Development
88323,"[Casual, Indie]",Beach Rules,2018-01-04,"[Casual, Indie, Pixel Graphics, Cute, 2D]",3.99,768880.0,Copperpick Studio


In [49]:
output_games['release_date'].unique()

array(['2018-01-04', '2017-07-24', '2017-12-07', ..., '2003-11-01',
       '2004-03-16', '2004-03-01'], dtype=object)

In [51]:
valores = ['SOON', 'SOON™','coming soon']

In [52]:
output_games = output_games[~output_games['release_date'].isin(valores)]

Se toma el año de la columna 'release data'

In [57]:
output_games['release_anio'] = output_games['release_date'].apply(lambda x: x.split('-')[0])

Se elimina la fecha depues de tomar solo el año

In [59]:
output_games.drop('release_date',axis=1,inplace=True)

In [61]:
output_games.head()

,genres,app_name,tags,price,id,developer,release_anio
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140.0,Kotoshiro,2018
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,"[Free to Play, Strategy, Indie, RPG, Card Game...",0,643980.0,Secret Level SRL,2018
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,"[Free to Play, Simulation, Sports, Casual, Ind...",Free to Play,670290.0,Poolians.com,2017
88313,"[Action, Adventure, Casual]",弹炸人2222,"[Action, Adventure, Casual]",0.99,767400.0,彼岸领域,2017
88315,"[Action, Adventure, Simulation]",Battle Royale Trainer,"[Action, Adventure, Simulation, FPS, Shooter, ...",3.99,772540.0,Trickjump Games Ltd,2018


Se modifica la columna price para que no hayan caracteres alfabeticos

In [62]:
output_games['price'] =output_games['price'].apply(lambda x: 0 if isinstance(x, str) else x)

In [63]:
output_games['price'].unique()

array([4.9900e+00, 0.0000e+00, 9.9000e-01, 3.9900e+00, 9.9900e+00,
       1.8990e+01, 2.9990e+01, 1.0990e+01, 2.9900e+00, 1.5900e+00,
       1.4990e+01, 1.9900e+00, 5.9990e+01, 8.9900e+00, 6.9900e+00,
       7.9900e+00, 3.9990e+01, 1.9990e+01, 7.4900e+00, 1.2990e+01,
       5.9900e+00, 2.4900e+00, 1.5990e+01, 1.2500e+00, 2.4990e+01,
       1.7990e+01, 6.1990e+01, 3.4900e+00, 1.1990e+01, 1.3990e+01,
       3.4990e+01, 1.4900e+00, 3.2990e+01, 9.9990e+01, 1.4950e+01,
       6.9990e+01, 1.6990e+01, 7.9990e+01, 4.9990e+01, 5.0000e+00,
       1.3980e+01, 2.9960e+01, 1.0999e+02, 1.4999e+02, 7.7171e+02,
       2.1990e+01, 8.9990e+01, 9.8000e-01, 1.3992e+02, 4.2900e+00,
       5.4990e+01, 6.4990e+01, 7.4990e+01, 8.9000e-01, 5.0000e-01,
       2.9999e+02, 1.2900e+00, 1.1999e+02, 4.4990e+01, 3.0000e+00,
       1.5000e+01, 5.4900e+00, 2.3990e+01, 4.9000e+01, 1.0930e+01,
       1.3900e+00, 3.6990e+01, 4.4900e+00, 2.0000e+00, 4.0000e+00,
       1.9500e+00, 1.5000e+00, 1.9900e+02, 1.8900e+02, 6.6600e

In [64]:
output_games.head()

,genres,app_name,tags,price,id,developer,release_anio
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140.0,Kotoshiro,2018
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,"[Free to Play, Strategy, Indie, RPG, Card Game...",0.00,643980.0,Secret Level SRL,2018
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,"[Free to Play, Simulation, Sports, Casual, Ind...",0.00,670290.0,Poolians.com,2017
88313,"[Action, Adventure, Casual]",弹炸人2222,"[Action, Adventure, Casual]",0.99,767400.0,彼岸领域,2017
88315,"[Action, Adventure, Simulation]",Battle Royale Trainer,"[Action, Adventure, Simulation, FPS, Shooter, ...",3.99,772540.0,Trickjump Games Ltd,2018


Se normanliza app_name y developer pcon la funcion title()

In [65]:
output_games['app_name']=output_games['app_name'].apply(lambda x: x.title())

In [66]:
output_games['developer']=output_games['developer'].apply(lambda x: x.title())

In [67]:
output_games[['id', 'price']] = output_games[['id', 'price']].astype({'id': 'int32', 'price': 'float16'})

In [68]:
output_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27445 entries, 88310 to 120443
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        27445 non-null  object 
 1   app_name      27445 non-null  object 
 2   tags          27445 non-null  object 
 3   price         27445 non-null  float16
 4   id            27445 non-null  int32  
 5   developer     27445 non-null  object 
 6   release_anio  27445 non-null  object 
dtypes: float16(1), int32(1), object(5)
memory usage: 1.4+ MB


In [69]:
output_games.to_csv('output_games_clean.csv', index=False)

AUTRALIAN_USERS_ITEMS

In [1]:
import gzip
import json
import ast

In [2]:
import pandas as pd

Comprimiendo el archivo json.

In [ ]:
with open('australian_users_items.json', 'rb') as f_in:
    with gzip.open('australian_users_items.json.gz', 'rt') as f_out:
        f_out.write(f_in.read())

Debido alimitaciones de memoria la base de datos, se crea un codigo que omita las lineas con error

In [3]:
data1 = []

with gzip.open(r"C:\Users\57319\Desktop\Henry\Proyecto_Individual\australian_users_items.json.gz", 'rt', encoding='utf-8') as file:
    for line in file:
        try:
            json_data = ast.literal_eval(line)
            data1.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

In [4]:
df_items = pd.DataFrame(data1)

In [5]:
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


Se guarda el dataframe en un csv

In [7]:
df_items.to_csv('items.csv', index=False)

In [ ]:
df_items=pd.read_csv('items.csv')

In [6]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [9]:
anidada_True(df_items)

"La/s columna/s con datos anidados es/son: ['items']"

In [10]:
df_items.shape

(88310, 5)

In [36]:
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


Se eliminana dos columnas descartables.

In [5]:
df_items.drop(['steam_id','user_url'],axis=1,inplace=True)

Ahora de desanida la columna items.

In [7]:
df_expand=df_items.explode('items')

In [8]:
df_expand.head()

,user_id,items_count,items
0,76561197970982479,277,"{'item_id': '10', 'item_name': 'Counter-Strike..."
0,76561197970982479,277,"{'item_id': '20', 'item_name': 'Team Fortress ..."
0,76561197970982479,277,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
0,76561197970982479,277,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
0,76561197970982479,277,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."


In [9]:
df_expand.to_csv('items_dict.csv', index=False)

In [10]:
df_descompuesto=pd.json_normalize(df_expand['items'])

In [11]:
df_descompuesto.drop(['item_name','playtime_2weeks'],axis=1,inplace=True)

In [12]:
df_final=pd.concat([df_expand.reset_index(drop=True), df_descompuesto.reset_index(drop=True)], axis=1)

In [13]:
df_users1=df_final.drop('items', axis=1)

In [14]:
df_users1.to_csv('items_final.csv', index=False)

In [15]:
df_users1.head()

,user_id,items_count,item_id,playtime_forever
0,76561197970982479,277,10,6.0
1,76561197970982479,277,20,0.0
2,76561197970982479,277,30,7.0
3,76561197970982479,277,40,0.0
4,76561197970982479,277,50,0.0


In [16]:
df_users1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   item_id           object 
 3   playtime_forever  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 157.8+ MB


In [17]:
df_users1.isnull().sum()

user_id                 0
items_count             0
item_id             16806
playtime_forever    16806
dtype: int64

Se descartan los items sin id de juego

In [18]:
df_users1.dropna(subset='item_id',inplace=True)

In [19]:
df_users1.isnull().sum()

user_id             0
items_count         0
item_id             0
playtime_forever    0
dtype: int64

In [20]:
df_users1=df_users1.astype({'item_id':'int32','playtime_forever':'float16', 'items_count':'int16'})

C:\Users\57319\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)


In [21]:
df_users1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5153209 entries, 0 to 5170013
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int16  
 2   item_id           int32  
 3   playtime_forever  float16
dtypes: float16(1), int16(1), int32(1), object(1)
memory usage: 117.9+ MB


In [ ]:
df_users1.to_csv('items_ready.csv', index=False)

Pasando los archivos definitivos a parket

In [2]:
reviews_ready=pd.read_csv('readywith_sentiment_analysis.csv')

In [4]:
games_ready=pd.read_csv('output_games_clean.csv')

In [6]:
users_ready=pd.read_csv('items_ready.csv')

In [7]:
reviews_ready.to_parquet(r'reviews.parquet', index= False)

In [8]:
games_ready.to_parquet(r'games.parquet', index=False)

In [9]:
users_ready.to_parquet(r'users.parquet', index=False)